# Forward Algorithm 

This is more computationally tractable than the brute-force approach, and works by caching intermediate values.




We compute this with 'cells'. Each cell of the forward algorithm trellis $\alpha_t(j)$ represents the probability of being in state $j$ after seeing the first $t$ observations, given the automaton $\lambda$.

The value of each cell $\alpha_t(j)$ is computed by summing over the probabilities of every path that could lead us to this cell.

Each cell expresses the following probability:

$$

\alpha_t(j) = P(o_1, o_2,...,o_t,q_t = j|\lambda)

$$



i.e. at time step $t$, $\alpha_t(j)$ is the probability of having seen the full sequence of observations 

$$
    \alpha_t(j) = \Sigma_{i=1}^N \alpha_{t-1}(i) a_{ij} b_j(o_t)
$$

$P(H|start) \times P(2|H) = 0.32 $ $

In [18]:
from markov import hot, cold, Temperature
from fractions import Fraction
from itertools import islice 
from typing import Iterable 

## Use the same weights as before

In [ ]:
from coefficients import A,B,PI

See Figure A.5

In [20]:
alpha_1_1 = PI[cold] * B[cold][3]
alpha_1_2 = PI[hot] * B[hot][3]

alpha_2_1 = (alpha_1_2 * A[hot][cold] + alpha_1_1 * A[cold][cold]) * B[cold][1]
alpha_2_2 = (alpha_1_2 * A[hot][hot] + alpha_1_1 * A[cold][hot]) * B[hot][1]


assert float(alpha_1_1) == 0.02, alpha_1_1
assert float(alpha_1_2) == 0.32, alpha_1_2
assert float(alpha_2_1) == 0.069, alpha_2_1
assert float(alpha_2_2) == 0.0404, float(alpha_2_2)



Note that this is a recursive algorithm: 
$$
    \alpha_t(j) = \Sigma_{i=1}^N \alpha_{t-1}(i) a_{ij} b_j(o_t)
$$

- (A.12)

Because $\alpha_t$ depends on $\alpha_{t-1}$

We can implement this in Python as a generator

In [26]:
from algorithms import forward_algorithm

In [28]:
observations = [3, 1, 3]
for observation, alpha in forward_algorithm(PI, A, B, observations):
    print(f"Observation: {observation} ice creams eaten.\tP(hot) = {float(alpha[hot]):.4f}, P(cold) = {float(alpha[cold]):.4f}")

Observation: 3 ice creams eaten.	P(hot) = 0.3200, P(cold) = 0.0200
Observation: 1 ice creams eaten.	P(hot) = 0.0404, P(cold) = 0.0690
Observation: 3 ice creams eaten.	P(hot) = 0.0235, P(cold) = 0.0051


Sanity check:

In [29]:
sequence = list(forward_algorithm(PI, A, B, observations))

assert sequence[0][1][hot] == alpha_1_2
assert sequence[0][1][cold] == alpha_1_1
assert sequence[1][1][hot] == alpha_2_2
assert sequence[1][1][cold] == alpha_2_1


### A longer sequence

In [30]:

observations = [1, 1, 1,1,1,1]
for observation, alpha in forward_algorithm(PI, A, B, observations):
    print(f"Observation: {observation} ice creams eaten.\tP(hot) = {float(alpha[hot]):.4f}, P(cold) = {float(alpha[cold]):.4f}")

Observation: 1 ice creams eaten.	P(hot) = 0.1600, P(cold) = 0.1000
Observation: 1 ice creams eaten.	P(hot) = 0.0292, P(cold) = 0.0570
Observation: 1 ice creams eaten.	P(hot) = 0.0092, P(cold) = 0.0201
Observation: 1 ice creams eaten.	P(hot) = 0.0031, P(cold) = 0.0069
Observation: 1 ice creams eaten.	P(hot) = 0.0011, P(cold) = 0.0023
Observation: 1 ice creams eaten.	P(hot) = 0.0004, P(cold) = 0.0008


So there's a 0.0004 chance of 'being on a hot day after eating 1 ice cream a day for 6 days'

## Finally


$$
P(O | \lambda) = \Sigma_{i=1}^N \alpha_T(i)
$$

In [31]:
observations = [3,1,3 ]
for i, (observation, alpha) in enumerate( forward_algorithm(PI,A,B,observations)):
    p = sum(alpha.values())

    print(f"i:{i}, P({observation}) {float(p):.4f}")

i:0, P(3) 0.3400
i:1, P(1) 0.1094
i:2, P(3) 0.0286
